In [1]:
import numpy as np
import matplotlib.pyplot as plt
import camb
from scipy.interpolate import interp1d
from classy import Class

%matplotlib inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'axes.labelsize' : 20})
plt.rcParams.update({'axes.grid' : False})

In [2]:
# import classy
# classy

### Original author for this updated notebook: Jamie Sullivan, github @jmsull

Matter power spectrum:

In [3]:
nuFlag=True
bolt_names = ['h','Omr','Omb','Omcb','ns','Y_p','Neff','smnu','As']

bolt_cosmo = [0.7,5.042e-5,0.046,0.224+0.046,1.0,0.24,3.046,0.06,1e-10*np.exp(3.043)] #this is the choice of Planck TTTEEE in class for nu
cosmo_dict = dict(zip(bolt_names,bolt_cosmo))
# cosmo_dict['Omm'] = cosmo_dict['Omcb']+cosmo_dict['smnu']/93.14/cosmo_dict['h']**2
cosmo_dict['Omcdm'] = cosmo_dict['Omcb']-cosmo_dict['Omb'] #need this line here or perturbations will fail
k = np.logspace(-4,1,2000)


In [4]:
class_ref = {
    'recombination': 'RECFAST',
    'tol_ncdm_bg':1.e-10,
    # 'recfast_Nz0':100000,
    'tol_thermo_integration':1.e-6,
    # 'recfast_x_He0_trigger_delta':0.01,
    # 'recfast_x_H0_trigger_delta':0.01,
    'k_min_tau0':0.002,
    'k_max_tau0_over_l_max':3.,
    'k_step_sub':0.015,
    'k_step_super':0.0001,
    'k_step_super_reduction':0.1,
    'start_small_k_at_tau_c_over_tau_h':0.0004,
    'start_large_k_at_tau_h_over_tau_k':0.05,
    # 'tight_coupling_trigger_tau_c_over_tau_h':0.005,
    # 'tight_coupling_trigger_tau_c_over_tau_k':0.008,


    'evolver':1,
    'tight_coupling_trigger_tau_c_over_tau_h':4.1e-4,
    'tight_coupling_trigger_tau_c_over_tau_k':6.1e-5,

    'start_sources_at_tau_c_over_tau_h':0.006,
    'l_max_g':50,
    'l_max_pol_g':50,
    'l_max_ur':50,
    'l_max_ncdm':50,
    'tol_perturbations_integration':1.e-6,
    'perturbations_sampling_stepsize':0.01,
    'l_logstep':1.026,
    'l_linstep':25,
    'hyper_sampling_flat':12.,
    'hyper_sampling_curved_low_nu':10.,
    'hyper_sampling_curved_high_nu':10.,
    'hyper_nu_sampling_step':10.,
    'hyper_phi_min_abs':1.e-10,
    'hyper_x_tol':1.e-4,
    'hyper_flat_approximation_nu':1.e6,
    'q_linstep':0.20,
    'q_logstep_spline':20.,
    'q_logstep_trapzd':0.5,
    'q_numstep_transition':250,
    'transfer_neglect_delta_k_S_t0':100.,
    'transfer_neglect_delta_k_S_t1':100.,
    'transfer_neglect_delta_k_S_t2':100.,
    'transfer_neglect_delta_k_S_e':100.,
    'transfer_neglect_delta_k_V_t1':100.,
    'transfer_neglect_delta_k_V_t2':100.,
    'transfer_neglect_delta_k_V_e':100.,
    'transfer_neglect_delta_k_V_b':100.,
    'transfer_neglect_delta_k_T_t2':100.,
    'transfer_neglect_delta_k_T_e':100.,
    'transfer_neglect_delta_k_T_b':100.,
    'neglect_CMB_sources_below_visibility':1.e-30,
    'transfer_neglect_late_source':3000.,
    'halofit_k_per_decade':3000.,
    'l_switch_limber':40.,
    'accurate_lensing':1,
    'num_mu_minus_lmax':1000.,
    'delta_l_max':1000.
}

# additional precision parameters for neutrinos only
class_ref_nu = {
    'radiation_streaming_approximation':2,
    'radiation_streaming_trigger_tau_over_tau_k':240.,
    'radiation_streaming_trigger_tau_c_over_tau':100.,
    'ur_fluid_approximation':3,
    'ur_fluid_trigger_tau_over_tau_k':50.,
    'ncdm_fluid_approximation':3,
    'ncdm_fluid_trigger_tau_over_tau_k':51.,
    'tol_ncdm_synchronous':1.e-10,
    'tol_ncdm_newtonian':1.e-10,
}

In [5]:
def run_camb(cosmo_dict,k,z,nu=False):
    cosmo_dict['Omcdm'] = cosmo_dict['Omcb']-cosmo_dict['Omb']

    pars = camb.CAMBparams()
    pars.set_cosmology(H0=cosmo_dict['h']*100,
                       ombh2=cosmo_dict['Omb']*cosmo_dict['h']**2,
                       omch2=cosmo_dict['Omcdm']*cosmo_dict['h']**2,
                       mnu=cosmo_dict['smnu'] if nuFlag else 0., 
                       num_massive_neutrinos=1 if nuFlag else 0,

                       YHe=cosmo_dict['Y_p'],
                       omk=0, tau=0.06) #fixed parameters, Planck neutrinos
    pars.set_accuracy(AccuracyBoost=3.0, lAccuracyBoost=3.0, DoLateRadTruncation=False)
    pars.InitPower.set_params(ns=cosmo_dict['ns'],
                              As=cosmo_dict['As'])

    pars.set_matter_power(redshifts=[0.])#, kmax=kmax/h) 
    pars.share_delta_neff = True 

    results = camb.get_results(pars)
    zmin,zmax = 0, 2
    nz=3
    camb_interp = camb.get_matter_power_interpolator(pars,zmin=zmin,zmax=zmax,nz_step=nz,nonlinear=False)
    pinterp = camb_interp.P #defaults #takes (z,k) pairs
    return pinterp(z,k), results

In [6]:
#should really just make a private toolbox with these functions somewhere...
def run_class(cosmo_dict,k,z,nu=False,pk=False,gauge='synchronous'):
    '''Simple call to boltzmann code using input cosmology parameter vector. k in h/Mpc.'''
    #setup
    if('h' not in cosmo_dict.keys()):
        h = cosmo_dict['H0']/100.
    elif('H0' not in cosmo_dict.keys()):
        cosmo_dict['H0'] = cosmo_dict['h']*100.
        h = cosmo_dict['h']
    if('Omcdm' not in cosmo_dict.keys()):
        cosmo_dict['Omcdm'] = cosmo_dict['Omcb']-cosmo_dict['Omb']
        # cosmo_dict['Omcdm'] = cosmo_dict['Omm']-cosmo_dict['Omb']
    c,G = 2.99792e5,4.30071e-9 
    h=cosmo_dict['h']
    if(nu): 
        #use Planck single massive neutrino if using neutrinos
        N_ncdm=1
        m_ncdm=0.06 
    else:
        N_ncdm=0
        m_ncdm=0 
    
    N_ur=3.046-N_ncdm
    # N_ur = 3.046
    print("Nur: {0:.3f}, Nncdm: {1:.3f}, mncdm: {2:.3f}".format(N_ur,N_ncdm,m_ncdm))
    
    ceng = Class()
    #gross but don't want to look up how to do it right now
    if(nuFlag): 
        ceng.set({'h':h,
              'omega_b':cosmo_dict['Omb']*h**2,
              'omega_cdm':cosmo_dict['Omcdm']*h**2,
              #'omega_r':cosmo_dict['Omr']*h**2,
              'n_s':cosmo_dict['ns'],
              'A_s':cosmo_dict['As'],
              'gauge':gauge,
              'N_ur': 2.0308,
              'N_ncdm': N_ncdm,
              'T_ncdm': 0.7133,  # IMPORTANT, see page 11 of arxiv:1104.2935
#               'T_ncdm': (4/11)**(1/3),  
              'm_ncdm': m_ncdm,
              'YHe': cosmo_dict['Y_p'],
              'ncdm_fluid_approximation':3,
              'z_reio': 7.6711
              })
    else: 
        ceng.set({'h':h,
              'omega_b':cosmo_dict['Omb']*h**2,
              'omega_cdm':cosmo_dict['Omcdm']*h**2,
              #'omega_r':cosmo_dict['Omr']*h**2,
              'n_s':cosmo_dict['ns'],
              'A_s':cosmo_dict['As'],
              'gauge':gauge,
              'N_ur': N_ur,
#               'N_ncdm': N_ncdm,
#               'm_ncdm': m_ncdm,
              'YHe': cosmo_dict['Y_p'],
              'ncdm_fluid_approximation':3,
              'z_reio': 7.6711,   
#               'k_per_decade_for_pk':4000 seems it is impossible to make tks give more ks???
              })
    print(m_ncdm)
    print(ceng.pars)
    #need lower z_max ow issues long complaint, need z<z_rec for pk to work apparently
    ceng.set(class_ref)
    ceng.set(class_ref_nu)
    ceng.set({'output':  'mPk, mTk','P_k_max_1/Mpc':10.0})
    ceng.compute()
    plin=[]
    for ki in k: plin.append(ceng.pk(ki*h,z)*h**3) 
    # get P(k) at redshift z=0
    transfers = ceng.get_transfer(z)
    ceng.struct_cleanup()
    ceng.empty()
        
    return transfers,plin,ceng


In [7]:
# bolt_cosmo = [0.7,5.042e-5,0.046,0.224+0.046,1.0,0.24,3.046,1e-10*np.exp(3.043)] #this is the choice of Planck TTTEEE in class for nu
# ts0,pL0,ceng = run_class(cosmo_dict,k[:10],np.exp(0)-1,nu=nuFlag,pk=True,gauge='synchronous')

In [8]:
ts0,pL0N,ceng = run_class(cosmo_dict,k,np.exp(0)-1,nu=nuFlag,pk=True,gauge='newtonian')


Nur: 2.046, Nncdm: 1.000, mncdm: 0.060
0.06
{'h': 0.7, 'omega_b': 0.022539999999999998, 'omega_cdm': 0.10976, 'n_s': 1.0, 'A_s': 2.096805313253679e-09, 'gauge': 'newtonian', 'N_ur': 2.0308, 'N_ncdm': 1, 'T_ncdm': 0.7133, 'm_ncdm': 0.06, 'YHe': 0.24, 'ncdm_fluid_approximation': 3, 'z_reio': 7.6711}


In [9]:
pCAMB, camb_results = run_camb(cosmo_dict,k,np.exp(0)-1,nu=nuFlag)


In [10]:
# #newtonian vs synch gauge
# plt.plot(k,pCAMB/pL0 - 1,label='Synchronous')
# plt.plot(k,pCAMB/pL0N - 1 ,label='Newtonian')
# plt.xscale('log')
# plt.ylabel(r'$\frac{P_{\rm{CAMB}}}{P_{\rm{CLASS}}} - 1$',fontsize=20)
# plt.xlabel(r'$k \ h/\rm{Mpc} $',fontsize=20)
# plt.axhline(0,ls=':',c='k')
# plt.legend()

In [11]:
# nuFlag=False
# tsnonuN,pL0nou,_ = run_class(cosmo_dict,k_dense,np.exp(0)-1,nu=nuFlag,pk=True,gauge='newtonian')
# _,pL0nouS,_ = run_class(cosmo_dict,k,np.exp(0)-1,nu=nuFlag,pk=True,gauge='synchronous')
# pCAMBnonnu,_ = run_camb(cosmo_dict,k,np.exp(0)-1,nu=nuFlag)


In [12]:
# #nonu comparison
# plt.title('nonu')
# plt.plot(k,pCAMBnonnu/pL0nouS,label='Synch')
# # plt.plot(k,pCAMBnonnu/pL0nou,label='Newt')
# plt.xscale('log')
# plt.ylabel(r'$\frac{P_{\rm{CAMB}}}{P_{\rm{CLASS}}}$',fontsize=20)
# plt.xlabel(r'$k \ h/\rm{Mpc} $',fontsize=20)
# plt.axhline(1,ls=':',c='k')
# plt.legend()
# plt.xlim(10**(-2.5),10**0)

# Perturbation time!





jms 6/15/22 - **NB** Had to change the CLASS source code (as EXPLICILTY DIRECTED TO by CLASS) to be able to get high-redshift perturbations to run. 
This is the no neutrino run

In [13]:
#Copied from class_public notebook - many_times - dropping the things needed to make their beautiful figure for brevity
#i.e. damping scale etc

#commented out values are the original notebook values from class_public
z_max_pk = 460000 #46000go to very high maximum redshift to see what is happening
k_per_decade = 400#40#     # number of k values, controls final resolution
k_min_tau0 = 40.       # this value controls the minimum k value in the figure (it is k_min * tau0)
P_k_max_inv_Mpc =1.0   # this value is directly the maximum k value in the figure in Mpc
tau_num_early = 2000#200#   # number of conformal time values before recombination, controls final resolution
tau_num_late = 200     # number of conformal time values after recombination, controls final resolution
tau_ini = 10#.1#          # first value of conformal time in Mpc

# Cosmological parameters and other CLASS parameters
# jms: 6/15/22 Below I commented out the parameters that I used before (some of which are no longer accepted)
common_settings = {# which output? transfer functions only
                   'output':'mTk, vTk',
                   'h':cosmo_dict['h'],
                   'omega_b':cosmo_dict['Omb']*cosmo_dict['h']**2,
                   'omega_cdm':cosmo_dict['Omcdm']*cosmo_dict['h']**2,
                   'n_s':cosmo_dict['ns'],
                   'gauge':'newtonian',
                   'N_ur': cosmo_dict['Neff'],#-1,
                   'N_ncdm': 0,
#                    'm_ncdm': 0.06,
                   'YHe': cosmo_dict['Y_p'],
                   'P_k_max_1/Mpc':10.0,
                   'z_max_pk':z_max_pk,
#                    'recfast_Nz0':z_max_pk*2.,
#                    'recfast_z_initial':z_max_pk+1.,
                   'k_per_decade_for_pk':k_per_decade,
                   'k_per_decade_for_bao':k_per_decade,
#                    'perturb_sampling_stepsize':'0.05',
#                    'ncdm_fluid_approximation':3,
                    #default is to use HyRec, but I want to see if the discrepancy I am seeing is due to HyRec
#                    'recombination':'RECFAST', #this seems to change nothing?
                   'z_reio' : 7.6711

                  }
 

In [14]:
# call CLASS 
M = Class()
M.set(common_settings)
M.set(class_ref) #When I ran this before, did not use the ref settings, using them now though...
M.set(class_ref_nu)

True

In [15]:
print(M.pars)

{'output': 'mTk, vTk', 'h': 0.7, 'omega_b': 0.022539999999999998, 'omega_cdm': 0.10976, 'n_s': 1.0, 'gauge': 'newtonian', 'N_ur': 3.046, 'N_ncdm': 0, 'YHe': 0.24, 'P_k_max_1/Mpc': 10.0, 'z_max_pk': 460000, 'k_per_decade_for_pk': 400, 'k_per_decade_for_bao': 400, 'z_reio': 7.6711, 'recombination': 'RECFAST', 'tol_ncdm_bg': 1e-10, 'tol_thermo_integration': 1e-06, 'k_min_tau0': 0.002, 'k_max_tau0_over_l_max': 3.0, 'k_step_sub': 0.015, 'k_step_super': 0.0001, 'k_step_super_reduction': 0.1, 'start_small_k_at_tau_c_over_tau_h': 0.0004, 'start_large_k_at_tau_h_over_tau_k': 0.05, 'evolver': 1, 'tight_coupling_trigger_tau_c_over_tau_h': 0.00041, 'tight_coupling_trigger_tau_c_over_tau_k': 6.1e-05, 'start_sources_at_tau_c_over_tau_h': 0.006, 'l_max_g': 50, 'l_max_pol_g': 50, 'l_max_ur': 50, 'l_max_ncdm': 50, 'tol_perturbations_integration': 1e-06, 'perturbations_sampling_stepsize': 0.01, 'l_logstep': 1.026, 'l_linstep': 25, 'hyper_sampling_flat': 12.0, 'hyper_sampling_curved_low_nu': 10.0, 'hyper

In [16]:
M.compute() #This takes a long time

In [17]:
#Copied from CLASS many_times.ipynv

# define conformal time sampling array
times = M.get_current_derived_parameters(['tau_rec','conformal_age'])
tau_rec=times['tau_rec']
tau_0 = times['conformal_age']
tau1 = np.logspace(np.log10(tau_ini),np.log10(tau_rec),tau_num_early)
tau2 = np.logspace(np.log10(tau_rec),np.log10(tau_0),tau_num_late)[1:]
tau2[-1] *= 0.999 # this tiny shift avoids interpolation errors
tau = np.concatenate((tau1,tau2))
tau_num = len(tau)

background = M.get_background() # load background table
#thermodynamics = M.get_thermodynamics() # load thermodynamics table

background_tau = background['conf. time [Mpc]'] # read conformal times in background table
background_z = background['z'] # read redshift
background_z_at_tau = interp1d(background_tau,background_z)

# check and inform user whether intiial arbitrary choice of z_max_pk was OK
max_z_needed = background_z_at_tau(tau[0])
if max_z_needed > z_max_pk:
    print( 'you must increase the value of z_max_pk to at least ',max_z_needed)
    () + 1  # this strange line is just a trick to stop the script execution there
else:
    print( 'in a next run with the same values of tau, you may decrease z_max_pk from ',z_max_pk,' to ',max_z_needed)

in a next run with the same values of tau, you may decrease z_max_pk from  460000  to  45570.849885589305


In [18]:
zz = np.zeros_like(tau) 
k_num = len( M.get_transfer(zz[0])['k (h/Mpc)']) #get number of k pts
pkeys = ['k (h/Mpc)', 'd_g', 'd_b', 'd_cdm', 'd_ur', 
#          'd_ncdm[0]', #drop for nonu
         'd_tot', 'phi', 'psi', 't_g', 't_b', 't_cdm', 't_ur', 
#          't_ncdm[0]',
         't_tot']
#get  all results
perts = np.zeros((len(pkeys),tau_num,k_num))
for i in range(tau_num):
    zz[i] = background_z_at_tau(tau[i])
    one_time = M.get_transfer(zz[i]) # transfer functions at each time 
    for s in range(len(pkeys)):
        perts[s,i,:] = one_time[pkeys[s]][:] 
        #the [:] is not necessay I don't think, is a view of the array
         #only use I have found so far for it is that a[:] = 10 assigns elementwise whereas a = 10 obviously does not
    
#flip things around to go from early to late times
zz=zz[::-1]
#convert z to x  #nice
xx = np.log(1/(1+zz))

In [ ]:
#get the slice first for a particular k
#this is lazy but just computing the full tau-k grid then slicing (we save the k value so it is fine to compare)
def get_kslice(ktarget, eps=0.01):
    k = perts[0,0,:]
    itarget = np.where((k>ktarget-eps) & (k<ktarget+eps))[0][0]
    return perts[:,:,itarget][:,::-1] 


In [ ]:
#save them
sliced_pert_1 = get_kslice(0.3,eps=0.001)
sliced_pert_2 = get_kslice(0.03,eps=0.001)
sliced_pert_3 = get_kslice(1.0,eps=0.001)

In [ ]:
#save the old modes
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp3_nofluid_nonu.dat',np.vstack([xx,sliced_pert_1]))
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp03_nofluid_nonu.dat',np.vstack([xx,sliced_pert_2]))
np.savetxt('../../Bolt/test/data/zack_N_class_px_k1p0_nofluid_nonu.dat',np.vstack([xx,sliced_pert_3]))

In [ ]:
#some more while we are here
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp1_nofluid_nonu.dat',np.vstack([xx,
                                                                                     get_kslice(0.1,eps=0.001)]))
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp5_nofluid_nonu.dat',np.vstack([xx,
                                                                                     get_kslice(.5,eps=0.01)]))
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp01_nofluid_nonu.dat',np.vstack([xx,
                                                                                     get_kslice(0.01,eps=0.001)]))
np.savetxt('../../Bolt/test/data/zack_N_class_px_kp001_nofluid_nonu.dat',np.vstack([xx,
                                                                                     get_kslice(0.001,eps=0.0001)]))

In [ ]:
thermo = M.get_thermodynamics()

In [ ]:
thermo.keys()

dict_keys(['scale factor a', 'z', 'conf. time [Mpc]', 'x_e', "kappa' [Mpc^-1]", 'exp(-kappa)', 'g [Mpc^-1]', 'Tb [K]', 'dTb [K]', 'w_b', 'c_b^2', 'tau_d'])

In [ ]:
np.savetxt('../../Bolt/test/data/zack_N_class_px_nofluid_nonu_x_e.dat',
np.vstack([thermo['scale factor a'],thermo["x_e"], thermo['Tb [K]'], thermo['c_b^2']]))